## 키워드 기반 추천시스템 recommendation_by_keyword
 - bookdata.csv에 있는 title, review을 활용하여 keyword을 입력하면 그 keyword들의 평균벡터와 가장 벡터값이 유사한 책을 추천
 - 작동 설명
     1. review에 있는 데이터들로 임베딩 모델을 만든 뒤 책마다 특정 벡터값을 가질 수 있도록 토큰화된 단어들의 벡터값의 평균을 낸다.
     2. 키워드를 인자로 입력받으면 그 키워드의 평균 벡터값을 내 문서 벡터마다 코사인 유사도를 구한다
     3. 구한 코사인 유사도가 높은 순서대로 5개의 책을 추천한다.

1. 데이터 로드

In [1]:
# 패키지 임포트

import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from konlpy.tag import Okt
import io
from io import BytesIO
!pip install gensim==3.7.0 --user
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 데이터를 데이터프레임으로 로드하고, 전체 문서의 수 출력

In [3]:
df = pd.read_csv('bookdata.csv')
print('전체 문서의 수 : ', len(df))

전체 문서의 수 :  139


In [4]:
# 상위 5행만 출력하여 확인

In [5]:
df[:5]

,title,author,isbn13,review
0,달러구트 꿈 백화점 2,이미예,9791165343729,달러구트 꿈백화점 1권을 읽으신 분들이라면 2권을 손꼽아 기다렸을 것이다. 나 또한...
1,달러구트 꿈 백화점,이미예,9791165341909,평소에 책을 너무 안 읽는 것 같아 계속 베스트셀러에 올라와 있기에 주문해 보았습니...
2,미드나잇 라이브러리,매트 헤이그,9791191056556,죽음이라는 주제로 어둡게 이야기가 흘러가지는 않을까 걱정했는데 죽음보다는 인생에 대...
3,"오늘 밤, 세계에서 이 사랑이 사라진다 해도",이치조 미사키,9791191043297,우연히 SNS에서 이 책의 줄거리를 읽게 되었습니다. 이런 표지의 감성을 담은 소설...
4,완전한 행복,정유정,9791167370280,무서워서 낮에만 읽으려고 했던 나의 결심?다짐?을 반달늪에 쳐박아 버린 소설. 정유...


In [6]:
# 형태소 분석 하기 전 사용자 사전 추가
from ckonlpy.tag import Twitter
twitter = Twitter()
twitter.add_dictionary(list(df['author']),'Noun')

C:\Users\user\AppData\Roaming\Python\Python38\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [7]:
# 'Desc' 열을 대상으로 전처리를 수행하여 'cleaned'에 저장

In [9]:
def _removeNonAscii(text):
    letters_only = re.sub('[^ 0-9가-힣]', '', text)   
    return letters_only

def remove_stop_words(text):
    twitter = Twitter()
    tokenized_data = []
    temp_X = twitter.nouns(text) # 토큰화
    
    # 5. Stopwords 불용어 제거    
    file=open('bool.txt','r',encoding='UTF8')
    stopwords=file.readlines()[0].split(' ')
    file.close()
    
    temp_X = [word for word in temp_X if not word in stopwords] 
    tokenized_data.append(temp_X)
    return( ' '.join(temp_X) )

df['cleaned'] = df['review'].apply(_removeNonAscii)
df['cleaned'] = df.cleaned.apply(remove_stop_words)

In [10]:
# 상위 5행만 출력하여 확인

In [11]:
df['cleaned'][:5]

0    달러 구트 꿈 백화점 권 분 라면 권 권 꿈 꿈 복권 꿈 자리 악몽 식은땀 다양 꿈...
1    평소 책 계속 베스트셀러 주문 판타지 내용 그냥 다가 계속 끝 부담 꿈 사고 판다 ...
2    죽음 주제 이야기 걱정 죽음 인생 루고 실제 죽음 삶 사이 경계 선 보기 주인공 삶...
3    우연 책 줄거리 표지 감성 소설 취향 줄거리 한번 보고 내용 궁금 구매 기억 남자 ...
4    낮 결심 다짐 반달 늪 박아 소설 정유 정 작가 소설 장소 버 린다 고 일어 모든 ...
Name: cleaned, dtype: object

In [12]:
# 빈 값이 있는 행 확인, nan 값으로 변환 후 해당 행 제거

In [13]:
df['cleaned'].replace('', np.nan, inplace=True)
df = df[df['cleaned'].notna()]
print('전체 문서의 수 :', len(df))

전체 문서의 수 : 139


In [14]:
# 토큰화하여 corpus 라는 리스트에 저장

In [15]:
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

2. Word2Vec을 통해 이를 초기 단어 벡터값으로 워드 임베딩

In [16]:
model = Word2Vec(corpus, size = 300, window=5, min_count = 5, workers = -1)
#word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True)
#word2vec_model.train(corpus, total_examples = word2vec_model.corpus_count, epochs = 15)

3. 단어 벡터의 평균 구하기

In [17]:
word2vec_model=model

def vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in word2vec_model.wv.vocab.keys():
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list 

document_embedding_list = vectors(df['cleaned'])
print('문서 벡터의 수 :',len(document_embedding_list))

<ipython-input-17-6bc9cd06868f>:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = word2vec_model[word]
<ipython-input-17-6bc9cd06868f>:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = doc2vec + word2vec_model[word]


문서 벡터의 수 : 139


In [18]:

def keyword_vector(keyword_list):    
    # 각 키워드에 대해서
    key2vec = None
    count = 0
    
    for word in keyword_list:
        if word in word2vec_model.wv.vocab.keys():
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if key2vec is None:
                    key2vec = word2vec_model[word]
                else:
                    key2vec = key2vec + word2vec_model[word]

    if key2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 키워드 개수로 나눠준다.
        key2vec = key2vec / count
            
    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return key2vec

4. 추천 시스템 구현하기

In [19]:
# 코사인 유사도를 이용하여, 가장 줄거리가 유사한 5개의 책을 찾아내는 함수 만들기

In [20]:
from sklearn.preprocessing import MinMaxScaler
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))


def recommendation_by_keyword(keyword_list):
    
    keyVector=keyword_vector(keyword_list)
    
    books = df[['title']]
    
    cosine_similarities2=[]
    for doc in document_embedding_list:
        cosine_similarities2.append(cos_sim(keyVector, np.array(doc)))
    

    # 입력된 키워드의 평균 벡트와 코사인 유사도가 가장 큰 책 5개 선정.
    sim_scores = list(enumerate(cosine_similarities2))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]

    # 가장 유사한 책 5권의 인덱스
    book_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = books.iloc[book_indices].reset_index(drop=True)

    # 데이터프레임으로부터 순차적으로 제목을 출력
    for i in book_indices:
        print(books.loc[i]['title']) 
    
    

In [21]:
# 확인 : 책 제목을 입력하면 이와 유사한 책들 추천(제목, 표지)

In [45]:
recommendation_by_keyword(['공포','암울'])

기억 2
유성의 인연 1
나무를 심은 사람
유성의 인연 2
아오이가든


<ipython-input-18-42c2dccf3232>:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  key2vec = word2vec_model[word]


In [39]:
recommendation_by_keyword(['삶','죽음'])

삶의 한가운데
바늘과 가죽의 시
사장님, 아무거나 먹지 마세요
너는 기억 못하겠지만
심판


<ipython-input-18-42c2dccf3232>:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  key2vec = word2vec_model[word]
<ipython-input-18-42c2dccf3232>:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  key2vec = key2vec + word2vec_model[word]


In [40]:
recommendation_by_keyword(['공포','스릴러'])

기억 2
그리스인 조르바
유성의 인연 1
한국단편소설 70
두근두근 내 인생


<ipython-input-18-42c2dccf3232>:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  key2vec = word2vec_model[word]
<ipython-input-18-42c2dccf3232>:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  key2vec = key2vec + word2vec_model[word]


In [47]:
recommendation_by_keyword(['로맨스','감동'])

우리가 함께 장마를 볼 수도 있겠습니다
달러구트 꿈 백화점 2
달러구트 꿈 백화점
사랑하라 한번도 상처받지 않은 것처럼
나는 내일, 어제의 너와 만난다


<ipython-input-18-42c2dccf3232>:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  key2vec = word2vec_model[word]
<ipython-input-18-42c2dccf3232>:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  key2vec = key2vec + word2vec_model[word]
